# 1. Importing Libraries

In [1]:
import cv2 #opencv
import numpy as np
import os   #helps with path
from matplotlib import pyplot as plt #to use plt.imshow()
import time              #to measure time between frames 
import mediapipe as mp   


# 2 . drawingutil and hands module

In [2]:
mpDraw=mp.solutions.drawing_utils
mpHands=mp.solutions.hands

# 3. Function to detect points on hands and then drawing on the hands

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB because by default opencv use bgr but we need rgb for mediapipe to process image
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            mpDraw.draw_landmarks(image,handsLms, mpHands.HAND_CONNECTIONS,
                             mpDraw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) )

# 4. function to get handedness ie left or right and to extract point to numpy array

In [5]:
def get_label(index,results):
    label = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            label = classification.classification[0].label
#         print(index,idx,label)
    if label:
        return label
    else:
        if index == 1:
            return get_label(0,results)
        elif index == 0:
            return get_label(1,results)
        else:
            return label
def extract_keypoints(results):
    hands = [np.zeros(21*3),np.zeros(21*3)]
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            label = get_label(num,results)
            if label == 'Right':
                hands[0] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            if label == 'Left':
                hands[1] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            
    return np.concatenate(hands)

# 5. Number of videos of dataset and videolength

In [6]:
# Thirty videos worth of data
no_videos = 200

# Videos are going to be 30 frames in length
video_length = 30

# 6. defining path of dataset and also path where we will store the processed data ie folder p1data in current directory

dataset link <a href="https://drive.google.com/drive/folders/1RZaXXy3pr7YLSv1jKFXDgmNC2nzkgV1P?usp=sharing">DATASET<a>

In [25]:
cwd = os.getcwd()
print(cwd)

C:\Users\91963\Desktop\HandSignRecognition\DEMO


In [8]:
datapath = os.path.join('KgData/asl4g/train/')
os.listdir(datapath)

['A',
 'B',
 'C',
 'D',
 'del',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'nothing',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'space',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [9]:
#path for numpy array of data
DATA_PATH = os.path.join('p1Data')

In [10]:
os.listdir(cwd)

['.ipynb_checkpoints',
 'HandGestureRecognition.ipynb',
 'HandGestureRecognitionNeuralNetwork.ipynb',
 'KgData',
 'model1.h5',
 'p1Data',
 'trainLogsdLstm']

# 7. defining the categories of signs 

In [7]:
categories = np.array(['A','B','C','D','E','F','G','H','I','J','del','space','nothing'])

# 8. making directory to store the processed data in p1Data folder

In [39]:
for category in categories: 
    for i in range(no_videos):
        try: 
            os.makedirs(os.path.join(DATA_PATH,category, str(i)))
        except:
            pass

In [25]:
# os.walk()

# 9. checking data is properly fetching  and also the camera feed

In [36]:
for category in categories:
    path = os.path.join(datapath,category)
    for imgpath in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,imgpath))
        img_array = cv2.resize(img_array,(480,640))
        img_array2 = cv2.flip(img_array,1)
        cv2.imshow("frame",img_array)
        cv2.imshow("frame2",img_array2)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture(0) #creating video capture object
## Set mediapipe model

while cap.isOpened():
    #reading feed current frame
    ret,frame = cap.read() 
    flipframe = cv2.flip(frame,1)
    cv2.imshow("Open Cv1",flipframe) 
    cv2.imshow('o2',frame)
    if cv2.waitKey(10) & 0xFF == ord('q'): #wait for 34 milli second to check if q is pressed on keyboard
        break 
cap.release() #release our webcame
cv2.destroyAllWindows()

# 10. Extracting keypoints using extract_keypoint funtion and for every frame   #  for every category we are creating 200 videos and every video contains 30 frames.

In [40]:
with mpHands.Hands() as model:
    for category in categories:
        path = os.path.join(datapath,category)
        adl = os.listdir(path)
        cdl = 0
        for i in range(no_videos):
            b = False
            for j in range(video_length):
                image = cv2.imread(os.path.join(path,adl[cdl]))
#                 image = cv2.flip(image,1)
                image = cv2.resize(image,(640,480))
                image, results = mediapipe_detection(image, model)
                draw_landmarks(image, results)
#                 print(results.multi_handedness)
                cdl += 1
                cv2.putText(image,'f no {} Vno {} c {}'.format(j,i,category), (10,20), 
                                   cv2.FONT_HERSHEY_SIMPLEX,0.5, (150, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow("frame",image)
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, category, str(i), str(j))
                np.save(npy_path, keypoints)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    b = True
                    break
            if b:
                break


    cv2.destroyAllWindows()

# 11. converting categories using one hot encoding and splitting data into train and test 

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
label_map = {label:num for num, label in enumerate(categories)}

In [27]:
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'del': 10,
 'space': 11,
 'nothing': 12}

In [28]:
videos, labels = [], []
for category in categories:
    for video in range(no_videos):
        window = []
        for frame_num in range(video_length):
            res = np.load(os.path.join(DATA_PATH, category, str(video), "{}.npy".format(frame_num)))
            window.append(res)
        videos.append(window)
        labels.append(label_map[category])

In [29]:
X = np.array(videos)

In [30]:
X.shape

(2600, 30, 126)

In [31]:
y = to_categorical(labels).astype(int) #one hot encoding to convert categorial variable

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [33]:
y_test.shape

(520, 13)

In [34]:
X_train.shape

(2080, 30, 126)

# 12. Building model and training

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Flatten
from tensorflow.keras.callbacks import TensorBoard

In [9]:
import tensorflow as tf
ACCURACY_THRESHOLD = 0.97
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # print(logs.get('acc'))
        # print(logs.get('categorical_accuracy'))
        if(logs.get('categorical_accuracy') > ACCURACY_THRESHOLD):
            # print(logs.get('acc'))
            # print(logs.get('categorical_accuracy'))
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()

In [10]:
log_dir = os.path.join('trainLogsdNeuralNetwork2')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
mycallbacks = [callbacks,tb_callback]

In [12]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(30,126)))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(categories.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [57]:
model.fit(X_train, y_train, epochs=500, callbacks=mycallbacks)

Epoch 1/500
65/65 [==============================] - 2s 25ms/step - loss: 2.0609 - categorical_accuracy: 0.2442
Epoch 2/500
65/65 [==============================] - 1s 12ms/step - loss: 1.4215 - categorical_accuracy: 0.4202
Epoch 3/500
65/65 [==============================] - 1s 11ms/step - loss: 1.1287 - categorical_accuracy: 0.5635
Epoch 4/500
65/65 [==============================] - 1s 10ms/step - loss: 0.8310 - categorical_accuracy: 0.6986
Epoch 5/500
65/65 [==============================] - 1s 10ms/step - loss: 0.6381 - categorical_accuracy: 0.7962
Epoch 6/500
65/65 [==============================] - 1s 10ms/step - loss: 0.5304 - categorical_accuracy: 0.8293
Epoch 7/500
65/65 [==============================] - 1s 10ms/step - loss: 0.6895 - categorical_accuracy: 0.7832
Epoch 8/500
65/65 [==============================] - 1s 11ms/step - loss: 0.4740 - categorical_accuracy: 0.8567
Epoch 9/500
65/65 [==============================] - 1s 11ms/step - loss: 0.5087 - categorical_accuracy:

65/65 [==============================] - 1s 10ms/step - loss: 0.1325 - categorical_accuracy: 0.9683
Epoch 144/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1317 - categorical_accuracy: 0.9683
Epoch 145/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1323 - categorical_accuracy: 0.9683
Epoch 146/500
65/65 [==============================] - 1s 10ms/step - loss: 0.1325 - categorical_accuracy: 0.9683
Epoch 147/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1317 - categorical_accuracy: 0.9683
Epoch 148/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1330 - categorical_accuracy: 0.9683
Epoch 149/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1322 - categorical_accuracy: 0.9683
Epoch 150/500
65/65 [==============================] - 1s 10ms/step - loss: 0.1316 - categorical_accuracy: 0.9683
Epoch 151/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1320 - categorical_accur

Epoch 215/500
65/65 [==============================] - 1s 13ms/step - loss: 0.1349 - categorical_accuracy: 0.9678
Epoch 216/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1343 - categorical_accuracy: 0.9678
Epoch 217/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1833 - categorical_accuracy: 0.9582
Epoch 218/500
65/65 [==============================] - 1s 13ms/step - loss: 0.6874 - categorical_accuracy: 0.8668
Epoch 219/500
65/65 [==============================] - 1s 14ms/step - loss: 0.2090 - categorical_accuracy: 0.9510
Epoch 220/500
65/65 [==============================] - 1s 22ms/step - loss: 0.1605 - categorical_accuracy: 0.9635
Epoch 221/500
65/65 [==============================] - 1s 17ms/step - loss: 0.1500 - categorical_accuracy: 0.9649
Epoch 222/500
65/65 [==============================] - 2s 29ms/step - loss: 0.1831 - categorical_accuracy: 0.9596
Epoch 223/500
65/65 [==============================] - 2s 25ms/step - loss: 0.3215 - cat

65/65 [==============================] - 1s 11ms/step - loss: 0.1311 - categorical_accuracy: 0.9683
Epoch 358/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1313 - categorical_accuracy: 0.9683
Epoch 359/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1315 - categorical_accuracy: 0.9683
Epoch 360/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1315 - categorical_accuracy: 0.9683
Epoch 361/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1314 - categorical_accuracy: 0.9683
Epoch 362/500
65/65 [==============================] - 1s 11ms/step - loss: 0.6601 - categorical_accuracy: 0.9495
Epoch 363/500
65/65 [==============================] - 1s 11ms/step - loss: 1.7560 - categorical_accuracy: 0.5933
Epoch 364/500
65/65 [==============================] - 1s 11ms/step - loss: 0.5683 - categorical_accuracy: 0.8221
Epoch 365/500
65/65 [==============================] - 1s 11ms/step - loss: 0.3132 - categorical_accur

Epoch 429/500
65/65 [==============================] - 1s 12ms/step - loss: 0.1505 - categorical_accuracy: 0.9644
Epoch 430/500
65/65 [==============================] - 1s 12ms/step - loss: 0.1398 - categorical_accuracy: 0.9678
Epoch 431/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1377 - categorical_accuracy: 0.9678
Epoch 432/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1358 - categorical_accuracy: 0.9678
Epoch 433/500
65/65 [==============================] - 1s 12ms/step - loss: 0.1346 - categorical_accuracy: 0.9678
Epoch 434/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1338 - categorical_accuracy: 0.9678
Epoch 435/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1338 - categorical_accuracy: 0.9683
Epoch 436/500
65/65 [==============================] - 1s 12ms/step - loss: 0.1326 - categorical_accuracy: 0.9683
Epoch 437/500
65/65 [==============================] - 1s 11ms/step - loss: 0.1329 - cat

In [58]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 30, 64)            8128      
_________________________________________________________________
dense_35 (Dense)             (None, 30, 64)            4160      
_________________________________________________________________
dense_36 (Dense)             (None, 30, 128)           8320      
_________________________________________________________________
dense_37 (Dense)             (None, 30, 256)           33024     
_________________________________________________________________
dense_38 (Dense)             (None, 30, 512)           131584    
_________________________________________________________________
dense_39 (Dense)             (None, 30, 512)           262656    
_________________________________________________________________
dense_40 (Dense)             (None, 30, 256)          

# 13. Make prediction 

In [59]:
res = model.predict(X_test)

In [60]:
categories[np.argmax(res[0])]

'nothing'

In [61]:
categories[np.argmax(y_test[0])]

'nothing'

# 14. Save Model

In [14]:
model.save('modelnn3.h5')

# Load Model

In [13]:
model.load_weights('modelnn2.h5')

# 15. Evaluation using Confusion Matrix and Accuracy


In [14]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [64]:
yhat = model.predict(X_test)

In [65]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [66]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[465,   2],
        [ 10,  43]],

       [[480,   2],
        [  2,  36]],

       [[477,   6],
        [  2,  35]],

       [[477,   3],
        [  4,  36]],

       [[484,   4],
        [  4,  28]],

       [[482,   1],
        [  2,  35]],

       [[482,   0],
        [  0,  38]],

       [[486,   0],
        [  2,  32]],

       [[473,   2],
        [  5,  40]],

       [[468,   0],
        [  3,  49]],

       [[483,   1],
        [  8,  28]],

       [[480,   4],
        [  7,  29]],

       [[454,  24],
        [  0,  42]]], dtype=int64)

In [16]:
accuracy_score(ytrue, yhat)

# 16. Testing in real Time

In [33]:
colors = [(245,117,16), (117,245,16), (16,117,245),(166,107,245),(106,167,245),(106,117,205),(161,107,25),(130,187,245),(106,117,205),(106,100,245),(196,107,245), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [34]:
back = cv2.imread('board.jpg')
back = cv2.resize(back,(800,800))
# cv2.imshow('back',back)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [16]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mpHands.Hands() as hand:
    while cap.isOpened():
        board = back
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame,hand)

        
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
#         print(sequence)
        sequence = sequence[-30:]
#         print(sequence)
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if categories[np.argmax(res)] != sentence[-1]:
                        sentence.append(categories[np.argmax(res)])
                else:
                    sentence.append(categories[np.argmax(res)])

            if len(sentence) > 10: 
                sentence = sentence[-20:]

            # Viz probabilities
            board = prob_viz(res, categories,board, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ''.join(sentence).replace('-',' '), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.imshow('viz',board)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'back' is not defined

In [18]:
cv2.destroyAllWindows()
cap.release()

In [ ]:
# sequence = []
# sentence = []
# threshold = 0.8
# with mpHands.Hands() as model:
#     for category in categories:
#         path = os.path.join(datapath,category)
#         adl = os.listdir(path)
#         cdl = 0
#         for i in range(no_videos):
#             b = False
#             for j in range(video_length):
#                 image = cv2.imread(os.path.join(path,adl[cdl]))
# #                 image = cv2.flip(image,1)
#                 image = cv2.resize(image,(640,480))
#                 image, results = mediapipe_detection(image, model)
#                 draw_landmarks(image, results)
# #                 print(results.multi_handedness)
#                 cdl += 1
#                 cv2.putText(image,'f no {} Vno {} c {}'.format(j,i,category), (10,20), 
#                                    cv2.FONT_HERSHEY_SIMPLEX,0.5, (150, 0, 255), 1, cv2.LINE_AA)
#                 cv2.imshow("frame",image)
#                 # NEW Export keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, category, str(i), str(j))
#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                     b = True
#                     break
#             if b:
#                 break


#     cv2.destroyAllWindows()

In [ ]:
#adding more layer to neural network